In [1]:
import tensorflow as tf
import numpy as np
import cv2

In [2]:
interpreter = tf.lite.Interpreter(model_path="lite-model_movenet_singlepose_lightning_3.tflite")
interpreter.allocate_tensors()

In [3]:
lowerjoints_list = [13,14,15,16]

In [4]:
def locating_joints(frame, confidence_threshold):
    for i in lowerjoints_list:
        y, x, conf = joints_coordinates[i]
        if conf > confidence_threshold:
            cv2.circle(frame, (int(x), int(y)), 4, (0,255,0), -1)

In [5]:
def knee_distance(frame):
    y1,x1 ,_ = joints_coordinates[13]
    y2, x2,_ = joints_coordinates[14]
    import math
    temp = (x2-x1)**2+(y2-y1)**2
    distance = math.sqrt(temp)
    cv2.putText(frame,"Distance:" + str(distance), (0,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
    
    

In [6]:
def left_slope(frame):
  import math
  y1,x1 ,_ = joints_coordinates[15]
  y2, x2,_ = joints_coordinates[13]
  m = (y2-y1)/(x2-x1)
  cv2.putText(frame,"left slope:" + str(m), (50,100), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  

In [7]:
def ankle_distance(frame):
    y1,x1 ,_ = joints_coordinates[15]
    y2, x2,_ = joints_coordinates[16]
    import math
    temp = (x2-x1)**2+(y2-y1)**2
    distance = math.sqrt(temp)
    cv2.putText(frame," Ank:" + str(distance), (150,200), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
    
    

In [8]:
def right_slope(frame):
  import math
  y1,x1 ,_ = joints_coordinates[16]
  y2,x2,_ = joints_coordinates[14]
  m2 = (y2-y1)/(x2-x1)
  cv2.putText(frame,"Right slope:" + str(m2), (100,150), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  

In [9]:
EDGES = {
    (13, 15): 'm',
    (14, 16): 'm',
    (13, 14): 'm',
    (16,15) : 'm'
}

In [10]:
def join_joints(frame, keypoints, edges, confidence_threshold):
    print(keypoints.shape)

    for edge, _ in edges.items():
        e1, e2 = edge
        y1, x1, conf1 = joints_coordinates[e1]
        y2, x2, conf2 = joints_coordinates[e2]
        
        
        if (conf1 > confidence_threshold) & (conf2 > confidence_threshold):      
           cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)


In [11]:
def insoles(frame): 
  y1,x1 ,_ = joints_coordinates[13]
  y2,x2,_ = joints_coordinates[14] 
  y3,x3 ,_ = joints_coordinates[15]
  y4,x4,_ = joints_coordinates[16]
  import math 
  dis = (x2-x1)**2+(y2-y1)**2
  dis = math.sqrt(dis)
  ankle_dis = (x4-x3)**2+(y4-y3)**2
  ankle_dis = math.sqrt(ankle_dis)
  print("ankle distance", ankle_dis)
  print("knee_distance:",dis)
  
  #cv2.putText(frame,"insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  if (dis <= 85.00000000000000 and dis > 73.000000000000 and ankle_dis <= 85.00000000000000  and ankle_dis > 73.000000000000) or (dis < 85.00000000000000  and dis > 80.00000000000000 and ankle_dis >= 85.00000000000000  and ankle_dis <= 88.00000000000000 ):
    print("running loop 1")
    cv2.putText(frame,"Medium arch height insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  elif (dis <= 73.000000000000 and ankle_dis <= 73.000000000000) or (dis < 73.000000000000 and ankle_dis > 73.000000000000 and ankle_dis <= 76.000000000000):
     #or (dis > 5608.855051436771 and ankle_dis < 7195.855191894665)
     print("running")
     cv2.putText(frame,"Low arch height insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  elif (dis > 85.00000000000000  and ankle_dis > 85.00000000000000 ):
     cv2.putText(frame,"High arch height insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  else:
    cv2.putText(frame,"Other medical condition", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
     #pass
            
   
    
    

In [12]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _, frame = cap.read()
    
    image = frame.copy()
    image = tf.image.resize_with_pad(np.expand_dims(image, axis=0),192,192)
    input_image = tf.cast(image, dtype=tf.float32)
    
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    print(output_details)
    
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    print("keypoints_with_scores")
    print(keypoints_with_scores)                
    y, x, c = frame.shape
    print("Denormalize keypoints with score")
    print(np.multiply(keypoints_with_scores, [y,x,1]))
    print("x , y and confidence threshold of keypoints")
    joints_coordinates = np.squeeze(np.multiply(keypoints_with_scores, [y,x,1]))
    print(joints_coordinates)
    
    locating_joints(frame, 0.4) 
    join_joints(frame, keypoints_with_scores, EDGES, 0.4)
    knee_distance(frame)
    ankle_distance(frame)
    left_slope(frame)
    right_slope(frame)
    insoles(frame)
    cv2.imshow('MoveNet', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

[{'name': 'StatefulPartitionedCall:0', 'index': 312, 'shape': array([ 1,  1, 17,  3]), 'shape_signature': array([ 1,  1, 17,  3]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
keypoints_with_scores
[[[[0.5597086  0.5386796  0.37579575]
   [0.4994535  0.59289247 0.59883165]
   [0.5006137  0.4839296  0.6772096 ]
   [0.5380198  0.6532649  0.6393188 ]
   [0.53803945 0.42360842 0.65448105]
   [0.8189745  0.8128225  0.7739186 ]
   [0.77083105 0.29506707 0.7335967 ]
   [0.96415395 0.8885902  0.05997777]
   [0.9330148  0.24134195 0.02157608]
   [0.8240901  0.70011973 0.00979239]
   [0.83335876 0.4808135  0.00556973]
   [1.0280144  0.8080012  0.00831899]
   [1.0137973  0.28988513 0.00777373]
   [0.84522784 0.85395634 0.03009397]
   [0.7759451  0.2809517  0.0136373 ]
   [0.5699798  0.9844846  0.01544994]
   [0.86892736 0.0150

In [13]:
cap.release()
cv2.destroyAllWindows()